## < 4. 멋진 작사가 만들기 >

## [루브릭]
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?

## Step1. 데이터 읽어오기

In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### glob
- glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
- 단, 조건에 정규식을 사용할 수 없으며 엑셀 등에서도 사용할 수 있는 ' * '와 ' ? '같은 와일드카드만을 지원
- glob를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장

---

## Step2. 데이터 정제
- 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

### preprocess_sentence()
1. 소문자로 바꾸고, 양쪽 공백을 지운다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꾼다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다
5. 다시 양쪽 공백을 지운다
6. 문장 시작에는 \<start>, 끝에는 \<end>를 추가한다

### re.sub（정규 표현식, 대상 문자열 , 치환 문자）
- 정규 표현식 : 검색 패턴을 지정
- 대상 문자열 : 검색 대상이 되는 문자열
- 치환 문자 : 변경하고 싶은 문자

In [3]:
# 정제된 문장을 넣을 리스트
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뛴다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    tmp = preprocess_sentence(sentence)

    # 길이가 15를 넘어가면 리스트에 넣지 않는다
    if len(tmp.split()) > 15 : continue 
    corpus.append(tmp)
    
        
    
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
def tokenize(corpus):
    
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus) 
    
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa0de15b2b0>


### tf.keras.preprocessing.text.Tokenizer( num_words=None, filters, lower=True, split=' ', char_level=False, oov_token=None,document_count=0, **kwargs)
- Tokenizer은 사전에 있는 단어의 순서에 맞게 단어를 turning하면서 말뭉치를 벡터화시킨다. 이때 숫자로 변환된 데이터를 텐서(tensor)라고 칭한다
    - num_words : 단어 빈도가 많은 순서로 num_words개의 단어를 보존한다
    - filters : 걸러낼 문자 리스트를 적어준다. 디폴트는 '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n' 이다
    - lower : 입력받은 문자열을 소문자로 변환할지를 True, False로 적어준다
    - split : 문자열을 적어 줘야 하고, 단어를 분리하는 기준을 적어준다
    - char_level : True인 경우 모든 문자가 토큰으로 처리된다
    - ov_token : 값이 지정된 경우, text_to_sequence 호출 과정에서 word_index에 추가되어 out-of-vocabulary words를 대체한다

### fit_on_texts()
- 문자 데이터를 입력받아서 리스트의 형태로 변환한다

### texts_to_sequences()
- 텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환한다

### tf.keras.preprocessing.sequence.pad_sequences()
- 시퀀스를 패딩을 하여 리스트의 길이를 동일하게 맞춰준다
- 파라미터 :
    - sequences: 리스트의 리스트로, 각 성분이 시퀀스
    - maxlen: 정수, 모든 시퀀스의 최대 길이를 설정하여 제한한다. 10을 넣으면 10보다 큰 것들은 자른다
    - dtype: 출력 시퀀스의 자료형. 가변적 길이의 문자열로 시퀀스를 패딩 하려면object를 사용하면 된다
    - padding: 문자열이 들어간다, 'pre'가 디폴트 값으로 앞쪽에 0이 추가되고, 'post'는 뒤쪽으로 0이 추가되어 각 시퀀스를 패딩 한다
    - truncating: 문자열, 'pre'는 길이가 초과됐을 때 앞쪽을 자르고 'post'는 maxlen보다 큰 시퀀스의 끝의 값들을 제거한다
    - value: 부동소수점 혹은 문자열, 패딩 할 값

In [5]:
# 단어 사전 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]   

---

## Step4. 평가 데이터셋 분리
- 단어장의 크기는 12,000 . 총 데이터의 20%를 평가 데이터셋으로 사용

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### tf.data.Dataset
- tf.data 는 데이터 입력 파이프 라인 빌드를 위한 텐서플로우의 서브패키지이다. 로컬 파일이나 메모리에 올려져 있는 데이터를 모델에 집어넣기 적합한 텐서로 변환하는 작업을 한다
- f.data.dataset 은 tf.data의 추상 클래스로써 데이터의 병렬 처리가 용이한 형태, 즉 GPU가 연산이 끝나면 다음 데이터를 바로바로 가져다가(Pre-Fetch) 빠르게 처리할 수 있도록 고안되었다. 
    - tf.data.Dataset.from_tensor_slices() : 주어진 데이터소스를 여러 Tensor로 자른다
    - dataset.shuffle(buffer_size) : dataset을 섞는다. buffier_size만큼 가져와서 섞는다.
    - dataset.batch(BATCH_SIZE, drop_remainder=True) : batch_size를 지정하여 size만큼 데이터를 읽어 들여 학습시킨다. 또한, model이 weight를 업데이트 할 때, 1개의 batch가 끝나고 난 후 업데이트를 하게 되는데, 업데이트 빈도를 조절하는 효과도 있다. drop_remainder는 마지만 남은 데이터를 drop 할 것인지 여부

In [11]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

---

## Step5. 인공지능 만들기
- val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계!

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

### model
- 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
    - Embedding 레이어 :
        - 텐서에는 단어 사전의 인덱스가 들어 있는데 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 준다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있다.
    - LSTM 레이어 :
        - hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가?로 생각하면 된다.
        - LSTM 레이어는 return_sequence 인자에 따라 마지막 시퀀스에서 한 번만 출력할 수도, 각 시퀀스에서 출력을 할 수도 있다. many to many 문제를 풀거나 LSTM 레이어를 여러 개로 쌓아올릴 때는 return_sequence = True 옵션을 사용한다.
        - 참고 : https://sevillabk.github.io/lstm-layer/

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.95445934e-04,  1.16711344e-04, -7.31838372e-05, ...,
          2.36492604e-04, -4.07708081e-04,  1.13148031e-04],
        [ 5.10004640e-04,  1.41668963e-04, -1.85336874e-04, ...,
          2.04536656e-04, -9.47582594e-04,  5.78537583e-04],
        [ 5.08022145e-04, -3.71952046e-05, -1.33670852e-04, ...,
          2.62836606e-04, -9.78183118e-04,  9.86682135e-04],
        ...,
        [-5.87882358e-04,  3.58387944e-04, -8.79816362e-04, ...,
          1.45049766e-03,  2.86493654e-04, -1.10942498e-03],
        [-2.31468235e-04,  5.00704336e-04, -1.33500493e-03, ...,
          2.09487928e-03,  7.42826494e-04, -9.95192677e-04],
        [ 1.18061682e-04,  7.56665366e-04, -1.76946772e-03, ...,
          2.71692662e-03,  1.14195689e-03, -8.31042300e-04]],

       [[ 2.95445934e-04,  1.16711344e-04, -7.31838372e-05, ...,
          2.36492604e-04, -4.07708081e-04,  1.13148031e-04],
        [ 1.16597475e-04, -2.25838099e-04,  3

###  shape=(256, 14, 12001)
- 256 : 지정한 배치 사이즈.
- 12001 : Dense 레이어의 출력 차원수
- 14 : LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력하는데, 데이터셋의 max_len이 14로 맞춰져 있었던 것

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  25174016  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=7, validation_data=val_dataset)

Epoch 1/7
487/487 [==============================] - 269s 548ms/step - loss: 3.2262 - val_loss: 2.8631
Epoch 2/7
487/487 [==============================] - 284s 583ms/step - loss: 2.6801 - val_loss: 2.6108
Epoch 3/7
487/487 [==============================] - 285s 585ms/step - loss: 2.3554 - val_loss: 2.4350
Epoch 4/7
487/487 [==============================] - 285s 585ms/step - loss: 2.0407 - val_loss: 2.2989
Epoch 5/7
487/487 [==============================] - 286s 586ms/step - loss: 1.7454 - val_loss: 2.2091
Epoch 6/7
487/487 [==============================] - 286s 587ms/step - loss: 1.4875 - val_loss: 2.1486
Epoch 7/7
487/487 [==============================] - 286s 587ms/step - loss: 1.2810 - val_loss: 2.1261


---

## Step6. 잘 만들어졌는지 평가하기

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20)

'<start> i like the flashy type who pass with dykes <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the only thing that keeps me goin <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> he is", max_len=20)

'<start> he is surrounded by the <unk> <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> she is", max_len=20)

'<start> she is a vacuum <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , i love you <end> '

---

## 회고

### 1. val_loss 를 2.2 이하로 낮추기 위한 과정

#### [embedding_size = 256, hidden_size = 1024]

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fl28E5%2FbtrrcGVFlgc%2FogmK8UyUBiKtTT34M0e4nK%2Fimg.png)

#### [embedding_size = 512, hidden_size = 1024]

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbZNJNZ%2FbtrrazQxf2P%2FLYEbHkVZZf06NgQbwxGnH1%2Fimg.png)

### 2. 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
- 이 프로젝트를 하면서 가장 고민을 많이 한 부분이다. 토큰화하는 과정에서 for문을 써서도 해보았고 tf.keras.preprocessing.sequence.pad_sequences() 인자에 maxlen=15을 넣어서도 해보았다. 결국 토큰화 하기전 corpus에 문장을 넣을 때, sentence의 길이가 15를 넘어가면 corpus에 넣지 않는 방법으로 처리를 해서 프로젝트를 진행했다. 아래의 사진을 보면 maxlen으로 처리했을 때랑 if문으로 처리했을 때랑 개수 차이가 많이나는걸 볼 수 있다. for문을 이용하면 남겨진 문장들은 문장의 끝이 있는 완성된 문장들이지만, maxlen을 이용하면 문장의 중간이 잘려나간 경우도 포함하게 된다. 이 부분에 대해서는 더 많은 생각이 필요할 것 같다!!

[maxlen=15]
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcQdZcs%2Fbtrq9WLXgf6%2FwdWVzYtuemxNF29jJkfkDK%2Fimg.png)

[if문]
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FrViis%2Fbtrrd7L1B8Y%2F3EKqNt8T3n1D6sTVFzDQ7k%2Fimg.png)

### 3. 과대적합
- embedding_size = 1024, hidden_size = 2048, epochs=10 으로 했을 때 epochs이 8일 때 val_loss가 다시 증가하는 현상이 발생했다. val_loss가 증가하는건 학습에 좋지 않고 과대적합이 발생했다 생각해서 과대적합이 발생하기 전까지 학습을 시켰다. 

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbowJFB%2Fbtrq7TWwPix%2FCjsCl4KPdVAsMT7tGTekUk%2Fimg.png)

### 4. 느낀점
- 이번 프로젝트가 NLP를 처음 접하게 된 프로젝트인데 국어를 좋아하지 않아서 그런지 지금까지 했던 프로젝트 중에서 가장 흥미를 느끼지 못한 프로젝트였다. 아직 한번밖에 접해보지 않았기 때문에 NLP에 대한 마음을 접으려고 하진 않지만 좀 더 해봐야 알 것 같다. 아직 optimizer나 loss에 대해 정확히 알지 못해서 이 부분에 대해서 더 많은 공부를 해야할 것 같고 토큰화에 대해서 좀 더 알아보려고 한다. 문장을 생성해보면 처음 생성된 문장은 존재하는 노래 가사가 그대로 나왔다. 그리고 she is a vacuum, 그녀는 진공 이라는 문장이 생성되었는데 이게 맞는 문장인지는 잘 모르겠다:) 그래도 문장이 만들어지는건 신기했다!!   